# Henry coefficient Calculations

In [1]:
using PorousMaterials

In [2]:
@eval PorousMaterials PATH_TO_CRYSTALS=joinpath(pwd(), "structural_relaxation", "post-relaxation_cifs")

"/home/ng/DTRA/structural_relaxation/post-relaxation_cifs"

In [ ]:
# # whtat properties do you want to put into the .csv?
# props_we_want = ["pressure (bar)", "fugacity (bar)", "⟨N⟩ (mmol/g)"]

# isotherm filename values
adsorbate = ["Kr", "Xe"] # : Xe, :Kr, :Ar

temp = 298.0 # units: K

pmin = -2 # minimum pessure in log10, units: bar
pmax = 1.1 # maximum pressure (actual value), units: bar
nstep = 15 # number of intervals in pressure range
pressures = 10 .^ range(pmin, stop=log10(pmax), length=nstep)

ffield = "UFF" # "UFF" or "Dreiding"

nburn = 50000 # number of burn cycles
nsample = 50000 # number of sample cycles

In [ ]:
# The name of the fragment
# file must be a .xyz file located in ./fragments/
# The atom species that is bonded to the
# :C_aro_R atom on the ring segment of the Fragment
fragment_list = [
#     "Br",
#     "C-3CH3",
#     "C-C",
#     "CH2-CH2-CH3",
#     "CH2-CH3",
#     "CH2-NH2",
#     "CH3",
#     "CH-CH2",
#     "CH-O",
#     "CH-S",
#     "Cl",
#     "C-N",
#     "F",
#     "N-2CH3",
#     "N-C-O",
#     "NH2",
#     "N-NH",
#     "O-CH2-CH2-CH3",
#     "O-CH2-CH3",
#     "O-CH3",
#     "O-C-N",
#     "OH",
#     "O-OH",
#     "P-2CH3",
#     "PH2",
#     "S-CH3",
#     "SH"
]


fragments_with_overlap = ["C-3CH3", "CH2-CH2-CH3", "CH2-CH3","CH2-NH2",
                          "CH-CH2", "CH-S", "N-2CH3", "O-CH2-CH2-CH3",
                          "O-CH3", "O-CH2-CH3", "P-2CH3", "PH2", 
                          "S-CH3", "SH"]

# The type of Arene Substitution ("ortho", "meta", "all") 
# TODO: "all" not yet implimented
# substitution_types = ["meta", "ortho"] 
substitution_types = ["meta"] 

In [ ]:
functional = "pbesol"
calculation = "relax"
n = length(fragment_list) * length(substitution_types)
xtals = Array{Crystal, 1} 
#####
# flag to determine whether xtals are going though a second stage of relaxation
#####
second_round = false

k = 1
for sub_type in substitution_types
    for fragment_name in fragment_list
        ####
        # set flag for overlaping atoms
        ####
        has_overlap = false
        if (fragment_name in fragments_with_overlap && !second_round)
            # pass flag for freezing parent atoms
            has_overlap=true 
            # indicate special case in filename
            crystal_name = "NiPyC2_relax_sc211_" * sub_type * "_functionalized_" *
                            fragment_name * "_" * functional * "_" * 
                            calculation * "_frozen_parent_atoms_Round2.cif"

        elseif (fragment_name in fragments_with_overlap && second_round)
            ####
            # Special Treatment for xtals going through 2nd round of relaxation ###
            ####
            crystal_name = "NiPyC2_relax_sc211_" * sub_type * "_functionalized_" *
                            fragment_name * "_" * functional * "_" * 
                            calculation * "_frozen_parent_atoms.cif"             
        else
            # determine name of crystal using rules from MOFun.jl
            crystal_name = "NiPyC2_relax_sc211_" * sub_type * "_functionalized_" * 
                                fragment_name * "_" * functional * "_" * calculation * ".cif"
        end
        # read in the crystal
        xtals[k] = Crystal(crystal_name)
        strip_numbers_from_atom_labels!(xtals[k])
        k += 1 # increase counter
    end
end

In [ ]:
# Calculate the Henry Coefficients for 
# the materials so that I can plot them

# henry_coefficient(crystal::Crystal, molecule::Molecule, temperature::Float64,
#                            ljforcefield::LJForceField; insertions_per_volume::Union{Int, Float64}=200,
#                            verbose::Bool=true, ewald_precision::Float64=1e-6,
#                            autosave::Bool=true, filename_comment::AbstractString="",
#                            accessibility_grid::Union{Nothing, Grid{Bool}}=nothing)


# save_name = henry_result_savename(crystal, molecule, temperature,
#                                    ljforcefield, insertions_per_volume;
#                                    comment="")
